In [27]:
import pyspark

import geopandas as gpd
import pandas as pd
import fiona
import rtree

sc = pyspark.SparkContext.getOrCreate()

In [2]:
TWEETS_FN = 'tweets-1m.csv'
DRUGS_FN1 = 'drug_illegal.txt'
DRUGS_FN2 = 'drug_sched2.txt'
CENSUS_TRACT_FN = '500cities_tracts.geojson'

In [3]:
tweets = sc.textFile(TWEETS_FN, use_unicode=True).cache()
#header = sc.broadcast({v: k for k, v in enumerate(taxi.first().split(','))})
#header.value
list(enumerate(tweets.first().split('|')))

[(0, '450845003775234048'),
 (1, '35.82840584'),
 (2, '-78.58814956'),
 (3, 'MarkaelDupree'),
 (4, 'Tue Apr 01 03:59:59 +0000 2014'),
 (5, '@RatedRnasty3_ haha hopefully !'),
 (6, 'haha hopefully')]

In [38]:
drugs = {line.strip() for line in open(DRUGS_FN1)} | {line.strip() for line in open(DRUGS_FN2)}
len(drugs)

3607

In [18]:
%time gdf = gpd.read_file('500cities_tracts.geojson').to_crs(fiona.crs.from_epsg(2263))
gdf.head()

CPU times: user 12.7 s, sys: 40 ms, total: 12.7 s
Wall time: 12.7 s


,plctract10,plctrpop10,geometry
0,0107000-01073005910,4612,(POLYGON ((-2901259.247383744 -2109114.1476620...
1,0107000-01073010801,168,(POLYGON ((-2916943.229076727 -2144480.3017811...
2,0107000-01073012701,44,(POLYGON ((-2884835.522685619 -2145459.4804165...
3,0107000-01073012703,498,(POLYGON ((-2884278.553546395 -2163666.8747638...
4,0107000-01073012704,113,(POLYGON ((-2884976.178560413 -2164988.8798753...


In [67]:
count = 0
for geom in gdf.geometry:
    if not geom.is_valid:
        count += 1
print (count)

200


In [68]:
len(gdf)

21346

In [69]:
gdf2 = gdf.loc[gdf.geometry.apply(lambda x: x.is_valid),:]
len(gdf2)

21146

In [70]:
count = 0
for geom in gdf2.geometry:
    if not geom.is_valid:
        count += 1
print (count)

0


In [71]:
count = 0
for pop in gdf2.plctrpop10:
    if not int(pop):
        count += 1
print (count)

880


In [21]:
gdf3 = gdf.loc[(gdf.plctrpop10.apply(lambda x: bool(int(x)))) & (gdf.geometry.apply(lambda x: x.is_valid)),:]
#https://stackoverflow.com/questions/35110632/splitting-self-intersecting-polygon-only-returned-one-polygon-in-shapely-in-pyth/35119152#35119152

In [25]:
gdf3.reset_index()

,index,plctract10,plctrpop10,geometry
0,1,0107000-01073010801,168,(POLYGON ((-2916943.229076727 -2144480.3017811...
1,3,0107000-01073012703,498,(POLYGON ((-2884278.553546395 -2163666.8747638...
2,5,0135896-01073012704,27,(POLYGON ((-2881277.422380826 -2167670.9543521...
3,8,0135896-01117030217,4878,(POLYGON ((-2885250.550574879 -2171205.3496426...
4,9,0135896-01117030303,2728,(POLYGON ((-2910024.639962874 -2178759.4991512...
5,10,0135896-01117030304,3288,(POLYGON ((-2932841.359172451 -2189037.8347076...
6,11,0137000-01089010612,2654,(POLYGON ((-2858879.82100596 -1696345.53729117...
7,12,0137000-01089010622,8725,(POLYGON ((-2832775.283959874 -1695159.7978084...
8,13,0137000-01089010800,860,(POLYGON ((-2778605.985282283 -1722931.5207731...
9,15,0150000-01097001502,1608,(POLYGON ((-3484487.422826784 -3119918.5861785...


In [80]:
count = 0
for geom in gdf3.geometry:
    if not geom.is_valid:
        count += 1
print (count)

0


In [81]:
count = 0
for pop in gdf3.plctrpop10:
    if not int(pop):
        count += 1
print (count)

0


In [ ]:
from shapely.geometry import LineString, MultiPolygon
from shapely.ops import polygonize, unary_union

#https://stackoverflow.com/questions/35110632/splitting-self-intersecting-polygon-only-returned-one-polygon-in-shapely-in-pyth/35119152#35119152
mls = unary_union(lr)
mp = MultiPolygon(list(polygonize(mls)))

In [63]:
%time gdf, ind = createIndex('500cities_tracts.geojson')

CPU times: user 18.7 s, sys: 130 ms, total: 18.8 s
Wall time: 18.8 s


In [26]:
import csv
count = 0
with open(TWEETS_FN) as f:
    reader = csv.reader(f, delimiter='|')
    for row in reader:
        if len(row) != 7:
            count += 1
print(count)

0


In [15]:
def distributeIndexFile(sc, zones, fn):
    from rtree.index import Index, Property
    idx, dat = '.idx', '.dat'
    p = Property(idx_extension=idx, dat_extension=dat)
    index = Index(fn, properties=p)
    for idx, geometry in enumerate(zones.geometry):
        index.insert(idx, geometry.bounds)
    index.close()
    sc.addFile(fn + idx)
    sc.addFile(fn + dat)
    
def readIndexFile(fn):
    import rtree
    return rtree.Rtree(fn)

def createIndex(geofile):
    import rtree
    import fiona.crs
    import geopandas as gpd
    zones = gpd.read_file(geofile).to_crs(fiona.crs.from_epsg(2263))
    index = rtree.Rtree()
    for idx, geometry in enumerate(zones.geometry):
        if geometry.is_valid and int(zones.plctrpop10[idx]):
            index.insert(idx, geometry.bounds)
    return (index, zones)

def findZone(p, index, zones):
    match = index.intersection((p.x, p.y, p.x, p.y))
    for idx in match:
        if zones.geometry[idx].contains(p):
            return idx
    return None

def processTweets(pid, records):
    import csv
    import pyproj
    import shapely.geometry as geom
    
    reader = csv.reader(records, delimiter='|')
    proj = pyproj.Proj(init='epsg:2263', preserve_units=True)
    
    ctracts = CTRACTS.value
    rtree_fn = RTREE_FN.value
    index = readIndexFile(rtree_fn)
    
    drug_words = {line.strip() for line in open('drug_illegal.txt')} | {line.strip() for line in open('drug_sched2.txt')}
    
    counts = {}
    
    for row in reader:
        if any(word in row[6] for word in drug_words):
            try:
                point = geom.Point(proj(float(row[2]),
                                        float(row[1])))
            except:
                continue
            ct = findZone(point, index, ctracts)

            if ct:
                match = (ctracts.plctract10[ct], int(ctracts.plctrpop10[ct])) 
                counts[match] = counts.get(match, 0) + 1
    
    return counts.items()

In [16]:
ctracts = gpd.read_file('500cities_tracts.geojson').to_crs(fiona.crs.from_epsg(2263))
    
#There are some invalid polygons, I'll get rid of them to avoid error when intersecting
#Also get rid of census tracts with 0 population
ctracts = ctracts.loc[(ctracts.plctrpop10.apply(lambda x: bool(int(x)))) & \
                      (ctracts.geometry.apply(lambda x: x.is_valid)),:]
CTRACTS = sc.broadcast(ctracts)

rtree_fn = 'ctracts_rtree'
RTREE_FN = sc.broadcast(rtree_fn)
distributeIndexFile(sc, ctracts, rtree_fn)

result = tweets.mapPartitionsWithIndex(processTweets) \
        .reduceByKey(lambda x,y: x+y) \
        .sortByKey() \
        .map(lambda x: ','.join([x[0][0], str(x[1]/x[0][1])])) \
        .collect()

TypeError: must be str, not int

In [60]:
result = tweets.mapPartitionsWithIndex(processTweets) \
            .reduceByKey(lambda x,y: x+y) \
            .map(lambda x: (x[0][0], x[1]/x[0][1]))
%time result.collect()

CPU times: user 20 ms, sys: 10 ms, total: 30 ms
Wall time: 3min 8s


[('0660466-06071002705', 0.005239666213856006),
 ('1210275-12071010410', 0.007500451834447858),
 ('3260600-32031001400', 0.01698867421718854),
 ('3651000-36005038800', 0.0049884881043745204),
 ('3251800-32003004602', 0.013613861386138614),
 ('3651000-36047055500', 0.021439509954058193),
 ('0669196-06083002403', 0.0014682131845543973),
 ('3601000-36001000404', 0.04478185662118009),
 ('0177256-01125012303', 0.004273504273504274),
 ('1245000-12086005407', 0.018876018876018877),
 ('1765000-17201002200', 0.008490930142802007),
 ('2255000-22071013400', 0.07293497363796134),
 ('1823278-18057110808', 0.0034810476295723284),
 ('3651000-36085002002', 0.004604051565377533),
 ('0566080-05143010302', 0.0012544273907910272),
 ('4159000-41051003301', 0.0214190093708166),
 ('2915670-29019001502', 0.004695623008552742),
 ('3651000-36005020501', 0.0047169811320754715),
 ('3651000-36081046900', 0.0004667444574095683),
 ('1225175-12001000200', 0.010900282599919257),
 ('2938000-29095016400', 0.001601708489

In [28]:
df = pd.read_csv('final_2.csv')
df.head()

,plctract10,drugtwts
0,0107000-01073000100,0.139053
1,0107000-01073000300,0.208044
2,0107000-01073000400,0.477831
3,0107000-01073000500,0.136173
4,0107000-01073000700,0.239038


In [31]:
gdf_final = gdf3.merge(df, on='plctract10').to_crs(fiona.crs.from_epsg(4326))
gdf_final.head()

,plctract10,plctrpop10,geometry,drugtwts
0,0107000-01073012703,498,"(POLYGON ((-86.628861 33.4765009999996, -86.62...",0.228916
1,0135896-01117030217,4878,"(POLYGON ((-86.628467 33.45578199999961, -86.6...",0.141246
2,0135896-01117030303,2728,"(POLYGON ((-86.704643 33.42567199999959, -86.7...",0.475073
3,0135896-01117030304,3288,"(POLYGON ((-86.773177 33.38893699999959, -86.7...",0.152676
4,0137000-01089010612,2654,"(POLYGON ((-86.76792 34.74868399999966, -86.76...",0.314619


In [32]:
gdf_final.to_file('final.geojson', driver='GeoJSON')